# Fachschaftssitzungen planen

Hier sind die Daten für die Fachschaftssitzungen in Form eines Python-Dictionary.
Um diese Daten zu ändern, muss man sie anklicken, modifizieren und dann mit _Shift + Enter_ die Änderungen bestätigen.
Damit die folgenden Zellen diese Daten dann auch verwenden, kann man das kleine _Play_-Symbol oben klicken und sieht immer, welche Zelle gerade berechnet wird, oder klickt auf das _Fast Forward_-Symbol um alle Zellen auszuführen.

_Achtung_: Wenn dieses Dokument auf mybinder.org läuft, dann werden die Daten dort nicht gespeichert.
Möchte man das Programm eventuell mehrmals laufen lassen, sollte man sich die folgende Eingabe irgendwo speichern und kann sie dann wieder an dieser Stelle einfügen.

Vor dem `:` ist der Name der Fachschaft, dahinter ein `set` (eine Menge) von Kürzeln für die Lehrer, die zu dieser Fachschaft gehören:

In [1]:
lehrer_in_fachschaft = {
    'Mathe': set(['Be', 'Ho']),
    'Physik': set(['Ho', 'Kp']),
    'Chemie': set(['Be', 'Fo']),
    'NWT': set(['Kp', 'Ho']),
    'Deutsch': set(['Fo', 'Li']),
}

Jetzt können wir alle möglichen Kombinationen finden, wann zwei Fachschaftssitzungen parallel stattfinden können.
Dazu wählen wir eine Fachschaft aus und testen, welche andere Fachschaft parallel tagen kann, weil keiner der Angehörigen der anderen Fachschaft in der ersten Sitzung war.

Das geht genau dann, wenn die Mengen der Teilnehmer disjunkt sind, also wenn die Schnittmenge leer ist; bspw.


In [2]:
lehrer_in_fachschaft['Mathe'].intersection(lehrer_in_fachschaft['Deutsch'])

set()

Diese Logik gießen wir in zwei Funktionen, mit denen wir einfach abfragen können, ob zwei oder mehrere Fachschaften gleichzeitig tagen können:

In [3]:
def kann_parallel_stattfinden(fachschaft1, fachschaft2):
    return lehrer_in_fachschaft[fachschaft1].intersection(lehrer_in_fachschaft[fachschaft2]) == set()

def kann_im_termin_stattfinden(fachschaften_im_termin, andere_fachschaft):
    for fachschaft in fachschaften_im_termin:
        if not kann_parallel_stattfinden(fachschaft, andere_fachschaft):
            return False
    return True

Es gibt mehrere verschiedene Möglichkeiten, wie man die Fachschaften parallel tagen lassen kann.
A priori ist keine davon _besser_ als eine andere.
Um das entscheiden zu können, müsste man eine _Metrik_ definieren.
So lange es so was nicht gibt, kann man aber mal ein paar verschiedene Möglichkeiten durchprobieren.

Im folgenden `for`-Loop iterieren wir über die verschiedenen Möglichkeiten, welche Fachschaft wir als _erste_ betrachten wollen.
Diese fügen wir als einzigen fest stehenden Teil in `termine` ein und entfernen sie gleichzeitig aus den `noch_zu_planen`den Fachschaften.
Danach gehen wir weiter durch die `noch_zu_planen`den Fachschaften und entfernen immer eine (mit `pop`), die wir `jetzt_verplanen` wollen.

Dann gehen wir durch die Termine, die wir bisher schon haben und schauen, ob die `jetzt_verplanen` Fachschaft parallel dazu stattfinden kann.
Wenn ja, fügen wir ihn in diese Terminschiene hinzu.
Wenn es nicht geht, dann starten wir eine neue Terminschiene damit.

_Achtung:_
An dem Algorithmus ist willkürlich

1. wie die `jetzt_verplanen` Fachschaft gewählt wird
2. in welcher Reihenfolge die termine abgelaufen werden (aktuell von "früh" nach "spät")

Beides müsste man mit einer _Heuristik_ oder _Metrik_ machen, um bessere von schlechteren Lösungen unterscheiden zu können.
(Eine _Low-Hanging-Fruit_ wäre, alles rauszuschmeißen, was die gleichen Kombinationen, nur in anderer Reihenfolge hat...)

In [4]:
fachschaften = list(lehrer_in_fachschaft.keys())

kombinationen = []

for nummer_erste_fachschaft in range(len(fachschaften)):
    termine = []
    noch_zu_planen = fachschaften.copy()
    erste_sitzung = noch_zu_planen.pop(nummer_erste_fachschaft)
    termine.append(set([erste_sitzung]))
    while noch_zu_planen:
        jetzt_verplanen = noch_zu_planen.pop()
        ist_verplant = False
        for termin in termine:
            if kann_im_termin_stattfinden(termin, jetzt_verplanen):
                termin.add(jetzt_verplanen)
                ist_verplant = True
                break
        if not ist_verplant:
            termine.append(set([jetzt_verplanen]))
    print("Wenn man als erste Fachschaft {} terminiert, dann kann man wie folgt planen: {}".format(
        fachschaften[nummer_erste_fachschaft], termine
    ))
    kombinationen.append(termine)

Wenn man als erste Fachschaft Mathe terminiert, dann kann man wie folgt planen: [{'Mathe', 'Deutsch'}, {'Chemie', 'NWT'}, {'Physik'}]
Wenn man als erste Fachschaft Physik terminiert, dann kann man wie folgt planen: [{'Physik', 'Deutsch'}, {'Chemie', 'NWT'}, {'Mathe'}]
Wenn man als erste Fachschaft Chemie terminiert, dann kann man wie folgt planen: [{'Chemie', 'NWT'}, {'Physik', 'Deutsch'}, {'Mathe'}]
Wenn man als erste Fachschaft NWT terminiert, dann kann man wie folgt planen: [{'Deutsch', 'NWT'}, {'Physik', 'Chemie'}, {'Mathe'}]
Wenn man als erste Fachschaft Deutsch terminiert, dann kann man wie folgt planen: [{'NWT', 'Deutsch'}, {'Physik', 'Chemie'}, {'Mathe'}]


Jetzt können wir diese Ergebnisse noch schön als Tabelle ausgeben:

In [5]:
def anzahl_benoetigter_schienen(termine):
    anzahl = 0
    for termin in termine:
        anzahl = max(anzahl, len(termin))
    return anzahl


from IPython.display import display, Markdown, Latex

variante = 1
for termine in kombinationen:
    schienen = anzahl_benoetigter_schienen(termine)
    text = 'Variante {}\n\n'.format(variante)
    variante += 1
    text += '| ' + ' | '.join(["Schiene {}".format(nr + 1) for nr in range(schienen)]) + ' |\n'
    text += '| --------- ' * schienen + '|\n'
    for schiene in termine:
        text += '| ' + ' | '.join([t for t in schiene])
        text += '   |' * (schienen - len(schiene))  # auffuellen, wenn zu wenige Fachschaften tagen
        text += ' |\n'
    text += '\n\n-------\n'
    display(Markdown(text))

Variante 1

| Schiene 1 | Schiene 2 |
| --------- | --------- |
| Mathe | Deutsch |
| Chemie | NWT |
| Physik   | |


-------


Variante 2

| Schiene 1 | Schiene 2 |
| --------- | --------- |
| Physik | Deutsch |
| Chemie | NWT |
| Mathe   | |


-------


Variante 3

| Schiene 1 | Schiene 2 |
| --------- | --------- |
| Chemie | NWT |
| Physik | Deutsch |
| Mathe   | |


-------


Variante 4

| Schiene 1 | Schiene 2 |
| --------- | --------- |
| Deutsch | NWT |
| Physik | Chemie |
| Mathe   | |


-------


Variante 5

| Schiene 1 | Schiene 2 |
| --------- | --------- |
| NWT | Deutsch |
| Physik | Chemie |
| Mathe   | |


-------
